In [1]:
# useing this variable for saving models and summaries
exp_name = "RCNN_Wmultitask_16_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

In [3]:
import models.CNN_lstm_model as CNN_lstm_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

pooling_stride = [4, 2, 2]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu

lstm_n_hidden = 128
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)

pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [9]:
pos_weight.shape

(19,)

In [10]:
pos_weight

array([  0.36565482,  48.46756152,  47.65126513,  50.0669746 ,
        49.94930876,  40.02411874,  41.52307692,  58.84303112,
        63.4664723 ,  75.37996546,  86.74603175,  80.14495413,
        87.27145709,  69.42038217,  68.86413902,  43.71587462,
        13.05721551,  58.2021419 ,  10.05323669])

In [11]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [12]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/' + exp_name 

In [13]:
model = CNN_lstm_model.Model(n_variables = n_variables,
                        n_classes = n_classes)

In [14]:
model.build_cnn_lstm_net(pooling_stride = pooling_stride,
                         weights_shape = weights_shape,
                         bias_shape = bias_shape,
                         activation = activation,
                         lstm_n_hidden = lstm_n_hidden,
                         n_classes = n_classes,
                         learning_rate = learning_rate,
                         pos_weight = pos_weight,
                         trainable_conv = True,
                         trainable_last = True,
                         variables_weights = variables_weights)

In [15]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [16]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.065 vs. 0.064
F1 = 0.103 vs. 0.099
Accuracy = 0.014 vs. 0.056
CHANGE DETECTION
AUC = 0.080 vs. 0.099
F1 = 0.192 vs. 0.183
Precision = 0.108 vs. 0.101
Recall = 0.907 vs. 0.953
Validation
ACTIVITY RECOGNITION
AUC = 0.061 vs. 0.060
F1 = 0.088 vs. 0.087
Accuracy = 0.012 vs. 0.056
CHANGE DETECTION
AUC = 0.036 vs. 0.048
F1 = 0.103 vs. 0.099
Precision = 0.055 vs. 0.052
Recall = 0.958 vs. 0.961
Minibatch Loss= 2.329664
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.711 vs. 0.061
F1 = 0.693 vs. 0.093
Accuracy = 0.648 vs. 0.056
CHANGE DETECTION
AUC = 0.258 vs. 0.094
F1 = 0.340 vs. 0.179
Precision = 0.260 vs. 0.099
Recall = 0.491 vs. 0.929
Validation
ACTIVITY RECOGNITION
AUC = 0.427 vs. 0.059
F1 = 0.486 vs. 0.081
Accuracy = 0.612 vs. 0.056
CHANGE DETECTION
AUC = 0.186 vs. 0.050
F1 = 0.278 vs. 0.097
Precision = 0.199 vs. 0.051
Recall = 0.460 vs. 1.000
Minibatch Loss= 0.284235
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.817 vs. 0.061


In [17]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/RCNN_Wmultitask_16_opp/RCNN_Wmultitask_16_opp-20000'

In [18]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
#saver.restore(sess, '../model/CNN7002_Gleam/CNN7002_Gleam-10000')


In [19]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.656 vs. 0.059
F1 = 0.667 vs. 0.082
Accuracy = 0.905 vs. 0.056
CHANGE DETECTION
AUC = 0.323 vs. 0.050
F1 = 0.372 vs. 0.096
Precision = 0.281 vs. 0.051
Recall = 0.553 vs. 0.992


In [20]:
out = nn_eval.deploy (model, sess, test_data, test_gt)

In [21]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(probability_of_pos, axis = 2)

In [24]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [25]:
1

1